# Generación de Variables Aleatorias Discretas

## Método de la Transformada Inversa

In [0]:
import random
import math

def probable(probabilidad):
    return probabilidad >= 0 and probabilidad <= 1

def aprox_uno(valor):
    return valor >= 0.99 and valor <= 1.01

In [0]:
def transformada_inversa(n_va, probabilidades):
    datos = sorted(probabilidades.items(), key=lambda x: x[1]) #Organiza de menor a mayor
    datos.reverse() #Revierte el orden para que esté organizado de mayor a menor
    
    resultados = [] #Almacena el resultado como una lista
    
    F_x = []
    suma_acum = 0
    
    for key, value in datos:
        suma_acum += value
        F_x.append(suma_acum)
        
    if(not aprox_uno(suma_acum)):
        raise Exception("El valor no asciende a 1")
        
    for i in range(n_va):
        num_random = random.random()
        contador = 0
        for i in F_x:
            if(num_random < i):
                key = "X_{}".format(datos[contador][0])
                resultados.append((key, num_random))
                break
            contador += 1
                
    return resultados

In [24]:
n_va = int(input("Número de Variables Aleatorias: "))
n_probabilidades = int(input("Número de probabilidades: "))

datos = {}
for i in range(n_probabilidades):
    key = i + 1
    valor = -1
    
    while(not probable(valor)):
        valor = float(input("Valor de p_{0}: ".format(key)))
        if(not probable(valor)):
            print("¡El valor no es una probabilidad!")
        
    datos[key] = valor

va =  transformada_inversa(n_va, datos)
for i in va:
    print(i)

Número de Variables Aleatorias: 2
Número de probabilidades: 2
Valor de p_1: 0.6
Valor de p_2: 0.4
('X_2', 0.7035216385438754)
('X_2', 0.8012863601985719)


## Método para la Generación de V.A. Poisson

In [0]:
def poisson(n_va, lambd):
    """
    
    """
    resultados = []
    for i in range(n_va):
        num_random = random.random()
        i = 0
        p = math.e ** (-1 * lambd)
        F = p
        while(num_random >= F):
            p = (lambd * p) / (i + 1)
            F = F + p
            i = i + 1

        key = "X_{}".format(i)
        resultados.append((key, num_random))
    
    return resultados

In [26]:
n_va = int(input("Número de Variables Aleatorias: "))
lambd = float(input("Valor de lambda: "))

va =  poisson(n_va, lambd)
for i in va:
    print(i)

Número de Variables Aleatorias: 2
Valor de lambda: 0.2
('X_0', 0.3432123023266753)
('X_0', 0.3779043886446032)


## Método para la Generación de V.A. Binomiales

In [0]:
def binomial(n_va, ensayos, probabilidad_sucesos):
    """
    """
    if(ensayos <= 0):
        raise Exception("El número de ensayos debe ser mayor o igual a cero")
    elif(not probable(probabilidad_sucesos)):
        raise Exception("El valor de probabilidad no está en el rango [0, 1]")
    
    resultados = []
    
    for i in range(n_va):
        num_random = random.random()

        probabilidad_fallos = 1 - probabilidad_sucesos

        c = probabilidad_sucesos / probabilidad_fallos
        i = 0
        pr = probabilidad_fallos ** ensayos
        F = pr

        while(num_random >= F):
            pr = (c * (ensayos - i) / (i + 1)) * pr
            F = F + pr
            i = i + 1

        key = "X_{}".format(i)
        resultados.append((key, num_random))
    
    return resultados

In [28]:
n_va = int(input("Número de variables aleatorias: "))
ensayos = int(input("Número de ensayos a realizar: "))
probabilidad_sucesos = float(input("Probabilidad de que el ensayo sea exitoso: "))

va = binomial(n_va, ensayos, probabilidad_sucesos)
for i in va:
    print(i)

Número de variables aleatorias: 2
Número de ensayos a realizar: 2
Probabilidad de que el ensayo sea exitoso: 0.3
('X_0', 0.0017295102495992198)
('X_1', 0.6728312314474237)


## Técnica de Aceptación y Rechazo para la Generación de V.A. Discretas

In [0]:
def calculate_c(list_p, list_q):
    if(len(list_p) != len(list_q)):
        raise Exception("La cantidad de valores en ambas listas debe ser igual.")
    
    tamaño_lista = len(list_p)
    division_list = []
    for i in range(tamaño_lista):
        division_list.append(list_p[i] / list_q[i])

    return max(division_list)


def operate_value(value, m):
    """
    """
    return int(m * value)
    

def admit_or_refuse_technique(list_p, list_q):
    """
    """
    if(len(list_p) != len(list_q)):
        raise Exception("La cantidad de valores en ambas listas debe ser igual.")
    
    result = []
    
    tamaño_lista = len(list_p)
    for i in range(tamaño_lista):
        simulated_y_value = 1
        num_random = 0
        p_y = 0
        q_y = 1
        c = 1
        
        while(num_random > p_y / (c * q_y)):
            simulated_y_value = operate_value(list_q[i], list_size)
            num_random = random.random()
            p_y = list_p[simulated_y_value]
            q_y = list_q[simulated_y_value]
            c = calculate_c(list_p, list_q)
        
        key = "X_{}".format(i + 1)
        result.append((key, simulated_y_value))
    
    return result

In [30]:
p_j = [0.1, 0.1, 0.1, 0.1, 0.6]
q_j = [0.4, 0.2, 0.1, 0.1, 0.2]
va = admit_or_refuse_technique(p_j, q_j)
for i in va:
    print(i)

('X_1', 1)
('X_2', 1)
('X_3', 1)
('X_4', 1)
('X_5', 1)


## Método de Composición para la Generación de V.A. Discretas

In [0]:
def composicion():
    num_random_1 = random.random()
    num_random_2 = random.random()
    
    resultados = []
    if(num_random_1 < 0.5):
        resultados.append(("X", int(10 * num_random_2)))
    else:
        resultados.append(("X", int(5 * num_random_2) + 6))
    return resultados

In [32]:
va = composicion()
for i in va:
    print(i)

('X', 5)


## Ejercicios 1, 3, 4 y 7 del capitulo 4

### Ejercicio 1
Escriba un programa para generar $n$ valores a partir de la función de masa de probabilidad $p_1 = \frac{1}{3}$, $p_2 = \frac{2}{3}$
- **_(a)_** Sea $n = 100$, ejecute el programa y determine la proporción de valores que sean iguales a 1.
- **_(b)_** Repita _(a)_ con $n = 1000$.
- **_(c)_** Repita _(a)_ con $n = 10000$.

In [33]:
def igual_uno(list):
    """
    """
    valor_igual_uno = 0
    for i in list:
        if("X_1" in i[0]):
            valor_igual_uno += 1
    return valor_igual_uno

probabilidades = {
    1: 1 / 3,
    2: 2 / 3
}
resultados_100 = transformada_inversa(100, probabilidades)
resultados_100 = igual_uno(resultados_100)
print("{} resultados son iguales a 1 para n = {}".format(resultados_100, 100))

resultados_1000 = transformada_inversa(1000, probabilidades)
resultados_1000 = igual_uno(resultados_1000)
print("{} resultados son iguales a 1 para n = {}".format(resultados_1000, 1000))

resultados_10000 = transformada_inversa(10000, probabilidades)
resultados_10000 = igual_uno(resultados_10000)
print("{} resultados son iguales a 1 para n = {}".format(resultados_10000, 10000))

28 resultados son iguales a 1 para n = 100
341 resultados son iguales a 1 para n = 1000
3339 resultados son iguales a 1 para n = 10000


### Ejercicio 3
Dé un algoritmo eficiente para simular el valor de una variable aleatoria X tal que
$$ P\{X=1\}=0.3, P\{X=2\}=0.2, P\{X=3\}=0.35, P\{X=4\}=0.15, $$

In [34]:
n_va = int(input("Número de Variables Aleatorias: "))
datos = {
    1: 0.3,
    2: 0.2,
    3: 0.35,
    4: 0.15
}
va =  transformada_inversa(n_va, datos)
for i in va:
    print(i)

Número de Variables Aleatorias: 2
('X_2', 0.6848557935510861)
('X_1', 0.5591728437602217)


### Ejercicio 7
Se lanza de manera continua un par de dados legales, hasta que todos los posibles resultados **2, 3,..., 12** hayan aparecido al menos una vez. Desarrolle un estudio de simulación para estimar el número esperado de lanzamientos necesarios.

In [35]:
def dados():
    dice_a = random.randint(1, 6)
    dice_b = random.randint(1, 6)
    return (dice_a, dice_b)


def suma_dados(dice_result):
    return dice_result[0] + dice_result[1]


def prueba():
    """
    Cuenta el número de lanzamientos
    necesarios para obtener todos los
    posibles resultados.
    """
    total = {} #almacenará los resultados como un diccionario.
    d = 0
    
    while(len(total) != 11): # 11 es el número total de posibles resultados
        resultados = suma_dados(dados())
        total[resultados] = True
        d += 1
    
    return d

resultados = []
lanzamientos = 10000
for i in range(lanzamientos):
    resultados.append(prueba())

promedido = sum(resultados) / lanzamientos
print("Se necesitan en promedio {} lanzamientos para obtener todos los posibles resultados".format(promedido))

Se necesitan en promedio 61.4216 lanzamientos para obtener todos los posibles resultados
